# 使用 Nsight Systems 优化简单 DNN 训练程序

## 5.1 主机和 GPU 之间的数据传输
主机和 GPU 设备之间的任何通信通常发生在速度非常慢的 PCIe 链路上，因此优化主机和 GPU 之间的任何数据传输非常重要。

## 5.2 分析配置文件
让我们分析第一个优化步骤中报告 (firstOptimization.nsys-rep) 中主机和 GPU 之间的数据传输。在 Nsight Systems GUI 中打开报告。通过单击前面的小三角形展开 `CUDA device row`。展开 `All Streams` 行以选择 `Memory` 行，然后右键单击以选择“在事件视图中显示”选项，如下所示。

<img src=images/ShowInEventsView.jpg>

这将填充“事件视图”窗口，其中按时间顺序列出内存操作。单击“持续时间”列标题按持续时间对事件视图中的表进行排序，以便最长的内存操作首先显示。右键单击表中的第一个条目，然后选择“在时间线上显示当前事件”，如下所示。

<img src=images/EventsView.jpg>

这会放大时间线上的事件，而青色高亮部分可帮助您找到在 GPU 上启动内存操作的 CUDA API 调用 `cudaMemcpyAsync`（见下图）。请注意，您可能需要缩小一点并向上滚动才能找到 CPU 线程上的 CUDA API 调用。

<img src=images/CudaMemcpyAsyncUsingPageableMemory.jpg>

我们从时间线上注意到以下情况：
- 所有主机到设备 (HtoD) 内存复制都使用可分页内存，导致 CPU 线程上的 `cudaMemcpyAsync` API 调用阻塞，直到操作在 GPU 上完成。
- 最长的 memcpy 操作在 GPU 上需要约 790 微秒才能完成。
- 最长的 memcpy 操作对应的 CUDA API 调用（`cudaMemcpyAsync`）大约需要 1ms。

## 5.3 优化代码以使用固定内存
主机 (CPU) 内存分配默认为可分页。GPU 无法直接从可分页主机内存访问数据。当从可分页主机内存向设备内存调用数据传输时，CUDA 驱动程序必须首先分配一个临时页面锁定（或“固定”）主机阵列，将主机数据复制到固定阵列，然后将数据从固定阵列传输到设备内存。固定内存用作从主机到设备的传输的暂存区。通过直接在固定内存中分配主机数据，我们可以避免这个额外的步骤及其开销。有关更多详细信息，请参阅[此博客](https://developer.nvidia.com/blog/how-optimize-data-transfers-cuda-cc/)。

<img src=images/PageableVsPinned.jpg width=50%>

我们程序中用于数据加载器 [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) 的设置依赖于 `pin_memory: False` 的默认值。执行下面的单元格以查看使用固定内存所做的代码更改。

In [ ]:
!diff -U4 --color=always application/main_opt1.py application/main_opt2.py

## 5.4 再次进行分析以验证优化
让我们通过执行下面给出的单元再次进行分析，以验证我们的代码更改是否解决了主机到设备内存传输的问题。

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--gpu-metrics-device=all \
--output /dli/task/nsys/secondOptimization \
--force-overwrite true \
python3 /dli/task/nsys/application/main_opt2.py

在 GUI 中打开报告 (secondOptimization.nsys-rep)。与我们在第 5.2 节中导航时间线的方式类似，展开 `CUDA device row` 及其下方的 `All Streams` 行。选择 `Memory` 行并右键单击以选择“在事件视图中显示”。按持续时间对事件视图中的表格进行排序，以使最长的内存操作首先显示。右键单击最上面的事件以选择“在时间线上显示当前事件”。您最终应该看到如下所示的视图。

<img src=images/Optimization2.jpg>

在优化后收集的配置文件中，我们观察到
- 所有 HtoD 内存复制现在都使用固定内存
- 最长的内存复制现在只有 511 微秒，而优化前为 790 微秒
- 与最长的内存复制相对应的 `cudaMemcpyAsync` API 调用现在只有约 20 微秒

现在我们已经解决了内存传输的瓶颈，让我们在 [步骤 06](06_tensor_core_util.ipynb) 中确定下一个性能瓶颈。